### 손글씨 데이터셋을 활용하여 CNN으로 학습 하고 분류하기
- 실제 손글씨데이터를 사용해도 높은 정확도가 나오는지 확인

In [ ]:
# 라이브러리 불러오기
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# 손글씨 데이터 불러오기
from tensorflow.keras.datasets import mnist

In [ ]:
(X_train, y_train), (X_test,y_test) = mnist.load_data()

11490434/11490434 [==============================] - 0s 0us/step


In [ ]:
# 정답데이터 확인해보기
# 10개의 클래스를 가진다 (0~9)

In [ ]:
# 훈련데이터 확인
X_train.shape

(1000, 28, 28, 1)

In [ ]:
# 훈련용데이터 1000개, 테스트용 데이터 300개
X_train = X_train[:1000, :]
y_train = y_train[:1000]

X_test = X_test[:300, :]
y_test = y_test[:300]

In [ ]:
# 정답데이터 : 범주형
y_train
# 예측데이터 : 확률값(클래스개수만큼)

In [ ]:
# 모델 설계시 필요한 라이브러리 불러오기
from tensorflow.keras.models import Sequential # 뼈대
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten

# Conv2D : 이미지특징 추출해주는 기능
# MaxPooling2D : 특징이 아닌 부분 제거
# Flatten : 다차원 데이터를 1차원으로 변경
# Dense : 분석

- 패딩
 - 필터의 크기로 인하여 가장자리 부분의 데이터가 부족, 입력과 출력의 이미지크기가 달라지는 것을 보완하기 위해서 입력데이터의 가장자리에 0을 미리 채워 놓는것
 - 입력과 출력의 크기를 맞춰줄수 있음
 - 가장자리에 있는 데이터의 학습량을 늘릴수 있다.

- same : 출력과 입력이 같아지도록 적절할 수의 패딩을 자동으로 입력

In [ ]:
X_train.shape
# 색상차원 추가하기 (흑백 1)
# 0~ 255 범위를 가지는 흑백의 범위를 0~1사이로 스케일링
X_train= X_train.reshape(X_train.shape[0], 28,28,1).astype('float32')/255
X_test= X_test.reshape(X_test.shape[0], 28,28,1).astype('float32')/255

In [ ]:
# 모델 설계
# 뼈대생성
model = Sequential()
# 특성추출부
model.add(Conv2D(filters = 32,
                 kernel_size=(3,3),
                 input_shape = (28,28,1),
                 padding = 'same',
                 activation = 'relu'
                 ))

# filters : 필터의 개수, 출력의 수
# kercel_size : 필터의 크기(일반적으로 (3,3), (5,5) 많이 사용함 )
# Input_shape : 입력데이터의 크기
# padding: cnn 단점을 보완하기위해 가장자리에 0값을 채워주는 역할
# activation : 활성화함수
model.add(MaxPooling2D(pool_size=(2,2)))
# pool_size : 축소비율 4배로 축소하겠다는 의미
# 4개의 픽셀중에서 가장두드러진 특징 1개만 빼고 나머지는 삭제

model.add(Conv2D(filters = 32,
                 kernel_size=(3,3),
                 padding = 'same',
                 activation = 'relu'
                 ))
model.add(MaxPooling2D(pool_size=(2,2)))

# 일반적으로 Con2D 2~3개정도 쓰고 Maxpooling2D 쓰는 경우가 많다!

# 특성 추출층: 2차원
# Dense 층 : 1차원의 데이터만 학습가능!
model.add(Flatten())

# 분석층
model.add(Dense(units= 128, activation='relu'))
# 위에서 이미 특징추출을 진행했기 때문에 분석층이 많이 필요하지 않음!

# 출력층 (다중분류)
model.add(Dense(units= 10, activation= 'softmax'))

In [ ]:
# 학습방법 및 평가방법 설정 (다중분류, 원핫인코딩X)
# loss, potimazer, metics
from tensorflow.keras.optimizers import Adam
model.compile( loss = "sparse_categorical_crossentropy",
                    optimizer = Adam(),
                     metrics = ['accuracy']
                     )

In [ ]:
# 모델 저장 (ModelCheckpoint) mc
# 모델 조기중단 (EarlyStopping) es
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

es = EarlyStopping(monitor="val_accuracy", verbose=1,patience=10)


model_path = "/content/drive/MyDrive/Colab Notebooks/Ai_Deep_Learning/data/mnist_cnn/mnist_cnn-{epoch:02d}-{val_accuracy:.2f}.hdf5"
mc = ModelCheckpoint(filepath=model_path,
                     verbose=1,
                        save_best_only=True,
                        monitor="val_accuracy")



In [ ]:
# 모델학습
h = model.fit(X_train, y_train, validation_split=0.2,
              epochs = 1000, batch_size= 50,
              callbacks= [mc,es])

Epoch 1/1000
14/16 [=========================>....] - ETA: 0s - loss: 2.0001 - accuracy: 0.3957  
Epoch 1: val_accuracy improved from -inf to 0.62000, saving model to /content/drive/MyDrive/Colab Notebooks/Ai_Deep_Learning/data/mnist_cnn/mnist_cnn-01-0.62.hdf5
16/16 [==============================] - 11s 21ms/step - loss: 1.9216 - accuracy: 0.4313 - val_loss: 1.3253 - val_accuracy: 0.6200
Epoch 2/1000
16/16 [==============================] - ETA: 0s - loss: 0.8131 - accuracy: 0.7775
Epoch 2: val_accuracy improved from 0.62000 to 0.77500, saving model to /content/drive/MyDrive/Colab Notebooks/Ai_Deep_Learning/data/mnist_cnn/mnist_cnn-02-0.77.hdf5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


16/16 [==============================] - 0s 9ms/step - loss: 0.8131 - accuracy: 0.7775 - val_loss: 0.7164 - val_accuracy: 0.7750
Epoch 3/1000
16/16 [==============================] - ETA: 0s - loss: 0.4914 - accuracy: 0.8413
Epoch 3: val_accuracy improved from 0.77500 to 0.82000, saving model to /content/drive/MyDrive/Colab Notebooks/Ai_Deep_Learning/data/mnist_cnn/mnist_cnn-03-0.82.hdf5
16/16 [==============================] - 0s 9ms/step - loss: 0.4914 - accuracy: 0.8413 - val_loss: 0.6273 - val_accuracy: 0.8200
Epoch 4/1000
12/16 [=====================>........] - ETA: 0s - loss: 0.3827 - accuracy: 0.8883
Epoch 4: val_accuracy improved from 0.82000 to 0.87000, saving model to /content/drive/MyDrive/Colab Notebooks/Ai_Deep_Learning/data/mnist_cnn/mnist_cnn-04-0.87.hdf5
16/16 [==============================] - 0s 9ms/step - loss: 0.3481 - accuracy: 0.8950 - val_loss: 0.5218 - val_accuracy: 0.8700
Epoch 5/1000
16/16 [==============================] - ETA: 0s - loss: 0.2506 - accuracy: 

In [17]:
from tensorflow.keras.models import load_model

best_model = load_model('/content/drive/MyDrive/Colab Notebooks/Ai_Deep_Learning/data/mnist_cnn/mnist_cnn-10-0.92.hdf5')

In [19]:
# test 데이터 활용하여 평가
best_model.evaluate(X_test, y_test)

10/10 [==============================] - 0s 12ms/step - loss: 0.2193 - accuracy: 0.9333


[0.21928395330905914, 0.9333333373069763]

In [21]:
# 파이썬에서 이미지를 처리하는 라이브러리
import PIL.Image as pimg
import numpy as np

# .convert("L") : 컬러이미지를 흑백이미지로 변환
img_2 = pimg.open('/content/drive/MyDrive/Colab Notebooks/Ai_Deep_Learning/data/손글씨/num3.png').convert("L")


In [22]:
# 이미지를 배열로 변환
img_2 = np.array(img_2)
img_2.shape


(28, 28)

In [29]:
# 원본데이터에서 진행했던 전처리를 그대로 진행
# 2차원 -> 1차원
img_2 = img_2.reshape(1,28,28,1).astype('float')/255
img_2.shape

(1, 28, 28, 1)

In [30]:
# 예측하기
best_model.predict(img_2)
# 부동소수점 : 뒤에 붙은 숫자만큼 소수점을 앞으로 이동시켜서 확인

1/1 [==============================] - 0s 149ms/step


array([[3.1112115e-05, 3.7155751e-04, 1.3386363e-02, 3.6624175e-01,
        5.0657537e-02, 6.4568431e-04, 2.6420455e-02, 1.1539303e-05,
        5.2828389e-01, 1.3950063e-02]], dtype=float32)

In [31]:
# 예측한 하나의 값만 확인하고 싶다면?
best_model.predict(img_2).argmax()

1/1 [==============================] - 0s 19ms/step


8